In [1]:
from sklearn.datasets import load_files
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
 
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

 
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [4]:
train_dir = 'fruits-360/Training'
test_dir = 'fruits-360/Test'
 
train2_dir = 'fruits-360/Training2'
test2_dir = 'fruits-360/test2'

In [5]:
#function which uses sklearn load_files to import the data
def load_db(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    
    

    
    
    
    targets = np.array(data['target'])
    img = np.copy(targets)
    
    labels = np.array(data['target_names'])
    return files, targets, labels

x_train, y_train, labels = load_db(train_dir)
x_test, y_test, labels = load_db(test_dir)

x_train2, y_train2, labels = load_db(train2_dir)
x_test2, y_test2, labels = load_db(test2_dir)
print('loading completed')

loading completed


In [6]:

#inds = img[:,:,2] < 100

In [7]:
#make sure classes are the same
no_of_classes = len(np.unique(y_train))
no_of_Test_classes = len(np.unique(y_test))
if no_of_classes == no_of_Test_classes:
    print(no_of_classes)
else:
    print("false")

false


In [6]:
#convert images to array
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array



x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)



x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

x_train2 = np.array(convert_image_to_array(x_train2))
print('Training set shape : ',x_train.shape)



x_test2 = np.array(convert_image_to_array(x_test2))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',x_train[0].shape)
print('1st training image shape ',x_test[0].shape)

Training set shape :  (3396, 100, 100, 3)
Test set shape :  (1140, 100, 100, 3)
Training set shape :  (3396, 100, 100, 3)
Test set shape :  (1140, 100, 100, 3)
1st training image shape  (100, 100, 3)
1st training image shape  (100, 100, 3)


In [7]:
x_test = x_test / 255.0
x_train = x_train / 255.0

x_test2 = x_test2 / 255.0
x_train2 = x_train2 / 255.0

In [8]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,no_of_classes)
y_test = np_utils.to_categorical(y_test,no_of_Test_classes)
y_train[0]
#only one class will result to 1 as that is true class for first image

array([0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [20]:
inputs = x_train.reshape(3896, 30000)
test_inputs = x_test.reshape(1635,30000)

In [17]:
NormalNN = Sequential()
    #Add the first layer to the neural network, it must take input of 1000 pixels as this is the size of our images
NormalNN.add(Dense(units = 512, activation = 'sigmoid', input_dim = 30000))
    #add a hidden layer
NormalNN.add(Dense(units = 512, activation = 'sigmoid'))
NormalNN.add(Dense(units=8, activation='softmax'))


NormalNN.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [18]:
NormalNN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               15360512  
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 4104      
Total params: 15,627,272
Trainable params: 15,627,272
Non-trainable params: 0
_________________________________________________________________


In [30]:
#Poor result got 1 in 10 categorical accuracy need to imporve this
batch_size = 32

history = NormalNN.fit(inputs,y_train,
        batch_size = batch_size,
        epochs=5,
        validation_split=0.2,
        verbose=1)

model_json = NormalNN.to_json()
print('saving model')
with open("normal_model.json", "w" ) as json_file:
    json_file.write(model_json)
model.save_weights("normal_model.H5")
print("model saved")

Train on 3116 samples, validate on 780 samples
Epoch 1/5
3116/3116 [==============================] - 19s 6ms/step - loss: 0.0624 - acc: 0.9955 - val_loss: 0.0709 - val_acc: 0.9949
Epoch 2/5
3116/3116 [==============================] - 19s 6ms/step - loss: 0.0412 - acc: 0.9981 - val_loss: 0.0685 - val_acc: 0.9962
Epoch 3/5
3116/3116 [==============================] - 19s 6ms/step - loss: 0.0741 - acc: 0.9958 - val_loss: 0.1044 - val_acc: 0.9846
Epoch 4/5
3116/3116 [==============================] - 19s 6ms/step - loss: 0.1453 - acc: 0.9663 - val_loss: 0.3577 - val_acc: 0.9192
Epoch 5/5
3116/3116 [==============================] - 19s 6ms/step - loss: 0.3092 - acc: 0.9150 - val_loss: 0.8942 - val_acc: 0.6615
saving model



In [32]:
# evaluate and print test accuracy
score = NormalNN.evaluate(test_inputs,y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.6403669725135197


## finished with normal neural network, Got good test accuracy now need to try using convolutional neural network

In [9]:
model = Sequential()
#model.add(GaussianNoise(0.1))
model.add(Conv2D(filters = 16, kernel_size = (3,3),input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GaussianNoise(0.1))


model.add(Conv2D(filters = 32,kernel_size = (3,3),activation= 'relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GaussianNoise(0.1))




model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(8,activation = 'softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)    

In [34]:

#compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['categorical_accuracy'])
print('Compiled!')

Compiled!


In [35]:
#number of images to take in in one go
batch_size = 64
#so model will save the best prediction
#need to take this out as weights generated are too large to push to heroku
#checkpointer = ModelCheckpoint(filepath = 'Fruit_image.hdf5', verbose = 1, save_best_only = True)


#Poor result got 1 in 10 categorical accuracy need to imporve this
history = model.fit(x_train,y_train,
        batch_size = batch_size,
        epochs=5,
        validation_split=0.2,
        verbose=1)

model_json = model.to_json()
print('saving model')
with open("cnn_model.json", "w" ) as json_file:
    json_file.write(model_json)
model.save_weights("CNN_model.H5")
print()

Train on 3116 samples, validate on 780 samples
Epoch 1/5
3116/3116 [==============================] - 48s 15ms/step - loss: 0.4368 - categorical_accuracy: 0.9188 - val_loss: 0.0523 - val_categorical_accuracy: 0.9974
Epoch 2/5
3116/3116 [==============================] - 49s 16ms/step - loss: 0.1364 - categorical_accuracy: 0.9917 - val_loss: 0.0389 - val_categorical_accuracy: 0.9974
Epoch 3/5
3116/3116 [==============================] - 50s 16ms/step - loss: 0.0870 - categorical_accuracy: 0.9939 - val_loss: 0.0252 - val_categorical_accuracy: 1.0000
Epoch 4/5
3116/3116 [==============================] - 50s 16ms/step - loss: 0.0596 - categorical_accuracy: 0.9974 - val_loss: 0.0180 - val_categorical_accuracy: 0.9962
Epoch 5/5
3116/3116 [==============================] - 51s 16ms/step - loss: 0.0447 - categorical_accuracy: 0.9987 - val_loss: 0.0154 - val_categorical_accuracy: 0.9974
saving model



### good test score but not valid when used on pictures

In [36]:
# evaluate and print test accuracy
score = model.evaluate(x_test,y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])



 Test accuracy: 0.9938837920489296


### realised this wont work for camera images so need to try another method

In [ ]:

train_datagen = ImageDataGenerator(
    rescale = 1/255.0,
    validation_split=0.2)

training_batches = train_datagen.flow_from_directory(train_dir,target_size=(100,100), class_mode='categorical', batch_size=32,subset='training')
valid_batches = train_datagen.flow_from_directory(train_dir,target_size=(100,100), class_mode='categorical', batch_size=32,subset='validation')

test_batches = train_datagen.flow_from_directory(train_dir,target_size=(100,100), class_mode='categorical', batch_size=32)




training_batches2 = train_datagen.flow_from_directory(train2_dir,target_size=(100,100),color_mode='rgba', class_mode='categorical', batch_size=32,subset='training')
valid_batches2 = train_datagen.flow_from_directory(train2_dir,target_size=(100,100),color_mode='rgba', class_mode='categorical', batch_size=32,subset='validation')

test_batches = train_datagen.flow_from_directory(train_dir,target_size=(100,100),color_mode='rgba', class_mode='categorical', batch_size=32)


all_training_batches2 = train_datagen.flow_from_directory(train2_dir,target_size=(100,100),color_mode='rgba', class_mode='categorical', batch_size=32)

In [25]:
model2 = Sequential()
#model.add(GaussianNoise(0.1))
model2.add(Conv2D(filters = 16, kernel_size = (3,3),input_shape=(100,100,4),padding='same'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(GaussianNoise(0.1))


model2.add(Conv2D(filters = 32,kernel_size = (3,3),activation= 'relu',padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(GaussianNoise(0.1))




model2.add(Flatten())
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(8,activation = 'softmax'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 100, 100, 16)      592       
_________________________________________________________________
activation_5 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
gaussian_noise_5 (GaussianNo (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 50, 50, 32)        128       
__________

In [26]:
#compile the model
model2.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['categorical_accuracy'])
print('Compiled!')

Compiled!


In [ ]:
# evaluate and print test accuracy
score = model2.evaluate(x_test,y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])

In [27]:
#we were using this checkpointer to save images, however heroku would not accept this file as it was too large so we 
#save the last weights instead
#checkpointer = ModelCheckpoint(filepath = 'fruit_image.H5', verbose = 1, save_best_only = True)



history = model2.fit_generator(all_training_batches2,
                    #class_weight=[1,1,1,1,1,1,1,0.20],
                    steps_per_epoch=1000, 
                    epochs = 2, 
                    validation_data=valid_batches2,
                    validation_steps = 1000,
                    verbose=1)
model_json = model2.to_json()
print('saving model')
with open("model2.json", "w" ) as json_file:
    json_file.write(model_json)
model2.save_weights("transparentBackground_cnn_model.h5")
print("Success")

Epoch 1/2
1000/1000 [==============================] - 772s 772ms/step - loss: 0.2336 - categorical_accuracy: 0.9432 - val_loss: 0.0281 - val_categorical_accuracy: 0.9939
Epoch 2/2
1000/1000 [==============================] - 772s 772ms/step - loss: 0.0674 - categorical_accuracy: 0.9827 - val_loss: 0.0222 - val_categorical_accuracy: 0.9935
saving model
Success


In [44]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
activation_3 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
gaussian_noise_3 (GaussianNo (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
batch_normalization_5 (Batch (None, 50, 50, 32)        128       
__________

In [45]:
# evaluate and print test accuracy
score = model2.evaluate_generator(test_batches,steps=1000, verbose=1)
print('\n', 'Test accuracy:', score[1])


1000/1000 [==============================] - 244s 244ms/step

 Test accuracy: 0.9942988534710848


### from the above it is clear to see we are going in the right direction, but I would still like to make it more accurate if possible

In [1]:
 
model3 = Sequential()
 
model3.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(100,100,4)))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.35))
 
 
model3.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model3.add(Conv2D(256, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.35))
 
model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.6))
model3.add(BatchNormalization())
model3.add(Dense(4, activation='softmax'))
 
model3.summary()
 
 
 
 
#compile the model
model3.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=.0005),
              metrics=['categorical_accuracy'])
print('Compiled!')
 


NameError: name 'Sequential' is not defined

In [ ]:
# evaluate and print test accuracy
score = model3.evaluate(x_test,y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])
